# DEMO notebook -- TEST version


### Set-up 
**Note:** In this notebook, the only class is **`Frame`** (**`Patterns`** and **`Groups`**, used in earlier notebooks, have been removed).

Before anything else, after having checked that the kernel you're running is a Python 3 kernel (it should say Python 3 on the top right, or see Help | About), open up the Git Bash prompt, and do the usual

```
git pull
```

This should pull everythig from GitHub. Then you're ready to run this notebook.

In [13]:
from framenet.data.annotation import get_frame_df, Frame, to_layers, gf_or_target, Groups
from framenet.data.pattern    import *
import pandas as pd, qgrid

## Overview

This notebook contains:

* a set of patterns used in the pattern-matching process
* Displays of valence patterns for a given FrameNet frame, filtered in various ways, and displayed as:
       * tables of valence patterns, with sentence examples
       * San-Key flow diagrams


## Patterns used for pattern-matching process 

We need to do this only once per Notebook. We need those becuase we describe the patterns we look for as Python expressions. The following applies the constructor `Lit` to the strings `'Ext'`, `'Obj'`, `'Dep'`, and `'_'` (our wildcard) and assigns the result to the corresponding Python variables. This way `Ext`, `Obj`, `Dep` and `x` can be combined in the patterns below.


   ### The symbols that are used in the pattern definitions:

<table>
<tr><th>Symbol</th><th>Meaning</th><th>Category</th></tr>                       
<tr><td>Ext</td><td>external argument (e.g. subject)</td><td>GF</td></tr>
<tr><td>x<td>wildcard element (e.g. verb)</td><td>Target</td></tr>
<tr><td>Obj</td><td>direct object</td><td>GF</td></tr>
<tr><td>Dep</td><td>dependent</td><td>GF</td></tr>
<tr><td>CNI</td><td>Constructional Null Instantiation</td><td>NI</td></tr>
<tr><td>DNI</td><td>Definite Null Instantiation</td><td>NI</td></tr>
<tr><td>INI</td><td>Indefinite Null Instantiation</td><td>NI</td></tr>
</table>



### [*QUESTION*: do we want to add more variables? If so, how?]


### The operators used to create patterns:

<table>
<tr><th>The operator</th><th>means</th></tr>                       
<tr><td>&amp;</td><td>_followed by_</td></tr>
<tr><td>`|`</td><td>_one or the other_</td></tr>
<tr><td>`Many1(...)`</td><td>_one or more of ..._</td></tr>
<tr><td>`Many(...)`</td><td>_zero or more of ..._</td></tr>
<tr><td>`Opt(...)`</td><td>_optionally ..._</td></tr>
</table>


In [14]:
Ext, Obj, Dep, x, CNI, DNI, INI = map(Lit, 'Ext Obj Dep _ CNI DNI INI'.split())

### Active transitive, no dependents

In [15]:
p1 = Ext & x & Obj 
p1_NI = Ext & x & Obj & Opt(CNI|DNI|INI)

### Active transitive, plus one or more dependents


In [16]:
p1_dep = Ext & x & Obj & Many1(Dep)
p1_dep_NI = Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)
p1_all = (p1 | p1_NI | p1_dep_NI)

### Intransitive or Passive transitive

In [17]:
p2 =  Ext & x 
p2_dep = Ext & x & Many1(Dep)
p2_dep_NI = Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)
p2_all = (p2|p2_dep_NI)

### Imperative for 'full' active transitive

This is pattern 'p1_dep_NI', minus the Ext argument

In [18]:
p3 = x & Obj & Many1(Dep) & Opt(Many(CNI|DNI|INI))
p3_obj = x & Obj & Opt(Many(CNI|DNI|INI))
p3_dep = x & Many(Dep) & Opt(Many(CNI|DNI|INI))
p3_all = (p3 | p3_obj | p3_dep)

### Active Transitive with Dep FE immediately following the verb
* E.g. 
Note that in FN annotation, bare prepositions are given the same PT as full prepositional phrases. 


In [19]:
p4 = Ext & x & Dep & Obj & Many(Dep) & Opt(CNI|DNI|INI)

### Active transitive in relative clauses

In [20]:
p5_rel_subj = Ext & Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)
p5_rel_obj  = Obj & (Obj) & Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)
p5_rel_subj_passive = Ext & Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)
p5_all = (p5_rel_subj | p5_rel_obj | p5_rel_subj_passive)

### One pattern to rule them all:

In [21]:
p_all = (p1_all | p2_all | p3_all |p4 |p5_all) 

## A table showing the patterns defined in this notebook:

<table>
<tr><th>Name</th><th>Desciption</th><th>Pattern</th><th>Finds</th></tr>                       
<tr><td>p1_all</td><td>**Active transitive declaratives**</td><td></td><td></td></tr>
<tr><td>p1</td><td>active transitive</td><td>Ext & x & Obj</td><td>Ext > V > Obj </td></tr>
<tr><td>p1_NI<td>active transitive plus optional NI FEs</td><td>Ext & x & Obj & Opt(CNI|DNI|INI)</td><td>Ext > V > Obj + (NI)</td></tr>
<tr><td>p1_dep</td><td>active transitive with dependent(s)</td><td>Ext & x & Obj & Many1(Dep)</td><td>Ext > V > Obj > Dep</td></tr>
<tr><td>p1_dep_NI</td><td>active transitive with dep(s), plus optional NI FEs</td><td>Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > V > Obj > Dep + (NI)</td></tr>
<tr><td>p2_all</td><td>**Intransitive/Passive transitive Declaratives**</td><td></td><td></td></tr>
<tr><td>p2</td><td>intransitive/active passive</td><td>Ext & x </td><td>Ext > Verb</td></tr>
<tr><td>p2_dep</td><td>intransitive/active passive with dep(s)</td><td>Ext & x & Many1(Dep)</td><td>Ext > Verb > Dep</td></tr>
<tr><td>p2_dep_NI</td><td>intransitive/active passive, plus optional NI FEs</td><td>Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Verb > Dep + (NI)</td></tr>
<tr><td>p3_all</td><td>**Imperatives** </td><td>x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>V > Obj > Dep</td></tr><tr><td>p3</td><td>Imperative active trans </td><td>x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>V > Obj > Dep</td></tr>
<tr><td>p4</td><td>active trans with v > dep > obj </td><td>Ext & x & Dep & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Dep > Obj > Many(Dep) + (NI)</td></tr>
<tr><td>p5_all</td><td>**Relative Clauses**</td><td></td><td></td></tr>
<tr><td>p5_rel_subj</td><td>active trans in subj relative clause </td><td>Ext & Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Ext> Obj > Many(Dep) + (NI)</td></tr>
<tr><td>p5_rel_obj</td><td>active trans in 0bj relative clause </td><td>Obj & (Obj) & Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Obj > Obj > Ext> Many(Dep) + (NI)</td></tr>
</table>

        
        

# Select FrameNet `Frame'   

'Frame` simply groups together the information related to a specific FN frame. 

In [22]:
cfm = Frame('Cause_fluidic_motion')

### Here are the lexical units for the selected frame:

In [23]:
cfm.groups.data_frame['annotationSet.LU'].unique()

array(['dribble.v', 'spill.v', 'spatter.v', 'splatter.v', 'splash.v',
       'spray.v', 'squirt.v', 'pump.v'], dtype=object)

# Look at different patterns: two methods 

## 1. Display method: show selected patterns as a table

### The `display` method understands the following keywords, all optional:

* `pattern_matcher`: defined as the above expressions, or `None` / unspecified. E.g. pattern_matcher = p1
* `min_count`: the minimum count for sentences matching a pattern; a nonnegative integer number
* `negative`: a boolean specifying whether a pattern should be inverted (when set to 'True', picks up sentences that *do not* match)

The display method also returns a total count of the sentences that meet the display criteria.  
One way to display all of the sentences annotated for a given frame is to use `display`without any parameters: `display()`


### Display example 1:  Look at all of the sentences annotated for the selected frame:

In [12]:
cfm.display()

### Display example 2: Show   'Active Transitive plus dependents' pattern (p1_dep) with more than one example

In [13]:
cfm.display(
    pattern_matcher=(p1_dep), 
    negative=False, 
    min_count=1,
   )

### Display example 4: Show examples after the pattern above (p1_dep) has been filtered out

In [14]:
cfm.display(pattern_matcher = p1_dep, min_count = 0, negative=False)

In [15]:
cfm.display(pattern_matcher = p4, min_count = 0, negative=False)

,freq.,Patterns
1,1,"Ext: Fluid (NP) → v → Dep: Manner (PP) → Obj: Area (NP) → INI: Cause (None)The rain , which had been coming and going , began to pour down again , spattering in big gobs all over the glass around them ."
2,1,Ext: Cause (NP) → v → Dep: Path (AVP) → Obj: Fluid (NP)This in turn helps your body to pump out toxins and excess fluid .
3,1,"Ext: Agent (NP) → v → Dep: Goal (AVP) → Obj: Fluid (NP) → Dep: Time (PP)Surprisingly , it 's normal for some women to squirt out a fluid at the moment of orgasm ."


In [16]:
cfm.display(pattern_matcher = p3, min_count = 0, negative=False)

,freq.,Patterns
1,4,"v → Obj: Fluid (NP) → Dep: Goal (PP) → CNI: Agent (None)Fat Trask choked with laughter , spattering half-chewed pasta over another man who slapped him upside the head .Splashing the liquid into a goblet , he carried the drink back to the bed and shoved it brusquely into Isabel 's hand .` When you 've got it full up to the top , splash a bit of Paraffin on the top and put the match to it .Simply spray Hot Shapes onto clean , dry hair before setting to get instant hold with a glossy finish ."
2,2,"v → Obj: Fluid (NP) → Dep: Goal (PP) → DNI: Agent (None)It was Barney who returned to it first , dribbling cream into his coffee and watching it swirl into a spiral on the surface .Beyond that , you can usually buy extensions covering accidental damage to the buildings , eg ; by putting your foot through the bedroom ceiling , or to the contents , eg , by spilling paint on a carpet ."


In [17]:
cfm.display(pattern_matcher = p5_rel_subj, min_count = 0, negative=False)

,freq.,Patterns
1,1,"Ext: Cause (NP) → Ext: Cause (NP) → v → Obj: Goal (NP) → Dep: Fluid (PP)During the engagement A.A. bursts were exploding all round me , my port plane received a near miss which made a large hole and spattered the aircraft with splinters , which injured my left elbow . """
2,1,Ext: Agent (NP) → Ext: Agent (NP) → v → Obj: Fluid (NP) → Dep: Goal (PP)A milkman has been temporarily blinded by two men who sprayed superglue into his eyes .
3,1,Ext: Agent (NP) → Ext: Agent (NP) → v → Obj: Fluid (NP) → Dep: Goal (PP) → Dep: Explanation (PP)This spotted sea hare ( above ) is a mollusc that can squirt a cloud of purple ink into the water in response to the approach of a predator .
4,1,"Ext: Agent (NP) → Ext: Agent (NP) → v → Obj: Fluid (NP) → Dep: Goal (PP) → Dep: Time (PPing)One thinks of the stories of child muggers bearing sharp knives , driven below by a greater police presence upstairs , of the Underground ` acid fiend "" who squirts hydrochloric acid on people 's legs before making his escape , of steamers , hip-hop-styled persons perpetrating economy-sized thefts ."


In [18]:
cfm.display(pattern_matcher = p5_rel_obj, min_count = 0, negative=False)

,freq.,Patterns
1,2,"Obj: Fluid (NP) → Obj: Fluid (NP) → Ext: Agent (NP) → v → Dep: Goal (PP) → Dep: Time (PP)Do n't forget to add the Aquasafe at this stage , to remove the chlorines and chloramines that our water boards are pumping into the tapwater at the moment .It was part of the defoliant Agent Orange , that the US forces sprayed on Vietnamese forests during the 1960s and 1970s ."


In [19]:
cfm.display(pattern_matcher = p_all, min_count = 0, negative=True)

,freq.,Patterns
1,2,"Dep: Time (AVP) → Ext: Fluid (NP) → v → Obj: Goal (NP) → INI: Cause (None)This time a few , reluctant drops spattered the millpond surface but did not disturb it .Then the monstrous green body exploded like a balloon of filth , spattering the walls of the cavern and the cages of the cowering young prisoners -- the last time he/she would set a mark upon them ."
2,1,"Obj: Fluid (NP) → Ext: Agent (NP) → v → Dep: Goal (PP)No matter how much Norse manure you splatter on to them , they 've a habit of coming up smelling of roses ."
3,1,"Ext: Fluid (NP) → Dep: Source (PP) → v → Obj: Goal (NP) → CNI: Agent (None)` Put it out , "" she screamed at Julie as another burst of fire from the UZI spattered the cottage ."
4,1,"Dep: Time (PP) → Ext: Goal (NP) → v → Dep: Fluid (PP) → Dep: Explanation (VPto) → CNI: Agent (None)In days of yore , such places were spattered with the blood of sacrificial victims to ensure good fortune in battle , and dunters are thought to be the spirits of these unhappy people ."
5,1,Ext: Goal (NP) → Dep: Manner (AVP) → v → Dep: Fluid (PP) → INI: Cause (None)Bloxham looked down to see that his silk tie was comprehensively spattered with puke .
6,1,Ext: Goal (NP) → Dep: Manner (AVP) → v → Dep: Fluid (PP) → CNI: Cause (None)Even at this distance I could see he was liberally spattered with yellow paint .
7,1,"Dep: Cause (PP) → Ext: Fluid (NP) → v → Obj: Goal (NP)It was night , and as the wind gusted down the iron chimney pipe , a shower of metal flakes spattered on to the wooden floor ."
8,1,Ext: Fluid (NP) → v → CNI: Cause (None) → INI: Goal (None)Rinse additive systems work by having a sensor linked to the rinse cycle actuator so that while rinse water is being pumped a metered dose of rinse additive is delivered .
9,1,"Ext: Agent (NP) → v → Obj: Goal (NP) → Dep: Manner (AVP) → Obj: Fluid (NP)We strap them to the plane 's fuselage and pump them full of compressed air until we achieve neutral buoyancy . """
10,1,Ext: Agent (NP) → Dep: Manner (AVP) → v → Obj: Fluid (NP) → Dep: Time (PPing) → INI: Source (None)They will probably pump more oil in coming months .


## 2. Flow diagram method: show selected patterns as a Sankey flow diagram

### The `select` method understands some of the same keywords as the  `display` method (above):

* `pattern_matcher`: defined as the above expressions, or `None` / unspecified. E.g. pattern_matcher = p1
* `negative`: a boolean specifying whether a pattern should be inverted (when set to 'True', picks up sentences that *do not* match)

The select method also returns a total count of the sentences that meet the display criteria.  

### It is also possible to specify whether 'noncore' FEs (frame elements) should be included in the flow diagram: 

* `diagram` method takes an optional keyword argument to select noncore `FE`s. The default value is `True`.
* the `diagram` method also takes an optional argument to specify diagram size. E.g. size=(960, 768)


### Note: this method ignore phrase type (PT) distinctions in the annotated data. 
          E.g. it groups together Dep: Path" regardless of whether the phrase type is PP or AVP.


### Flow diagram method 1: Chart for all the valence patterns in the frame

In [24]:
cfm.select(p_all, negative=False).diagram(noncore=False, size=(480, 320))

Written 59 records.


<IPython.core.display.Javascript object>

In [25]:
cfm.select(p1_dep, negative=False).diagram(noncore=False, size=(480, 320))

Written 13 records.


<IPython.core.display.Javascript object>

In [26]:
cfm.select(p3_all, negative=False).diagram(noncore=False, size=(480, 320))

Written 9 records.


<IPython.core.display.Javascript object>

In [27]:
cfm.select(pattern_matcher=x, negative=False).diagram(noncore=True, size=(640, 480))

Written 131 records.


<IPython.core.display.Javascript object>

In [24]:
cfm.select(p_all, negative=True).diagram(noncore=False, size=(960, 768))

Written 8 records.


<IPython.core.display.Javascript object>

## More examples we worked on

### The `Filling` frame

Comment here.

In [25]:
filling = Frame('Filling')
filling.groups.data_frame['annotationSet.LU'].unique()

array(['adorn.v', 'anoint.v', 'cover.v', 'dust.v', 'load.v', 'pack.v',
       'smear.v', 'spread.v', 'stuff.v', 'wrap.v', 'plaster.v', 'drape.v',
       'dab.v', 'daub.v', 'inject.v', 'cram.v', 'sow.v', 'seed.v',
       'brush.v', 'hang.v', 'spatter.v', 'splash.v', 'splatter.v',
       'spray.v', 'sprinkle.v', 'squirt.v', 'shower.v', 'drizzle.v',
       'heap.v', 'pile.v', 'pump.v', 'scatter.v', 'butter.v', 'asphalt.v',
       'surface.v', 'tile.v', 'wallpaper.v', 'coat.v', 'suffuse.v',
       'fill.v', 'strew.v', 'douse.v', 'flood.v', 'crowd.v', 'pave.v',
       'varnish.v', 'paint.v', 'gild.v', 'glaze.v', 'embellish.v',
       'panel.v'], dtype=object)

In [26]:
filling.display(pattern_matcher=p1_dep_NI, min_count=0, negative=False)

In [27]:
filling.select(p1_dep_NI).diagram(noncore=False)

Written 4 records.


<IPython.core.display.Javascript object>

In [28]:
filling.display(pattern_matcher=p5_all, negative=False)

,freq.,Patterns
1,5,"Ext: Goal (NP) → Ext: Goal (NP) → v → Dep: Theme (PP) → CNI: Agent (None)Last week the Government warned that around 300 women who 'd been injected with a hormone called gonadotrophin , extracted from the pituitary glands of human corpses , could be at risk .They have thus become the animal equivalents of the plants which are force-fed with artificial fertilizers and sprayed with herbicides and pesticides .Virions were pelleted by centrifugation at 40,000 rpm for 1 hr , resuspended in 100µl 2% phosphotungstic acid and dropped onto Formvar-coated grids which had previously been coated with a thin layer of carbon .The process takes a previously printed image which before the ink is dry is dusted with a resinous powder .Their names , kept secret until election night , are written upon slips of paper which are wrapped in wax paper before being sealed in hot wax ."
2,3,"Obj: Goal (NP) → Obj: Goal (NP) → Ext: Agent (NP) → v → Dep: Theme (PP)She led him through to the dining-room and found him a glass of whisky and a plate which she proceeded to heap with food .Norman opened his tool-box and took out a pair of rubber gloves , which he dusted with talcum powder , and drew over his sensitive digits .My appearance , which Helmut had embellished with jewellery , expensive clothes and regular visits to beauty salons , Jean-Claude liked unadorned and without make-up ."
3,2,"Ext: Agent (NP) → Ext: Agent (NP) → v → Obj: Goal (NP) → Dep: Theme (PP)During the engagement A.A. bursts were exploding all round me , my port plane received a near miss which made a large hole and spattered the aircraft with splinters , which injured my left elbow . ""We chose the area for its sessions of powerful westerly winds , winds which crowd nearby Porto Pollo with Europe 's top windsurfers ."
4,1,"Ext: Goal (NP) → Ext: Goal (NP) → v → Dep: Agent (PP) → INI: Theme (None)That was a hint to Chris , who was sitting beside the basket that had been packed by Molly Fletcher ."
5,1,Ext: Agent (NP) → Ext: Agent (NP) → v → Obj: Goal (NP)I met there Mr Philpot who will paint the ceiling in my bedroom … ’


In [29]:
filling.display(pattern_matcher=p3_obj, negative=False)

,freq.,Patterns
1,10,"v → Obj: Goal (NP) → CNI: Agent (None) → INI: Theme (None)Obviously Marshall , apart from daubing one of the walls and putting up a piece of cork-board , had n't done anything .Always cover a sandpit or the local cats will use it as a toilet .Anointing the shoulders or arms was to symbolize the emperor 's protective and defensive role .In bathrooms , it 's advisable to tile walls other than those immediately above the bath -- excessive condensation looks unattractive and can shorten the life of the product .The West Midlands County Council will be left with the job of surfacing the roads .Nearly £15,000 has been spent on landscaping and surfacing the area by The Globe public house and down to Alresford Pond .When finance became available to complete the park in a traditional gardenesque manner , there was so much opposition that , apart from surfacing a few paths and strengthening the natural scrub , it was left .Today even the most fervent opponents of union with Canada admit that the move brought its blessings , including money to pave the roads and all the benefits of a welfare state .Could we suggest paving the area around West Hall War Memorial .The stone has now vanished and is believed to have been used for paving the walks of St. James Church in Brackley ."
2,8,"v → Obj: Goal (NP) → CNI: Agent (None)In both cases , seed the filters by running them in an existing balanced set-up for a couple of days before a buying trip .Then , when a drop of solution is removed from that test-tube to seed the next test-tube , it will be the new mutant variety that does the seeding .Butter a shallow ovenproof dish or 4–5 individual soufflé dishes .Butter the bread , spread peanut butter on four slices and mayonnaise on the other four .When wallpapering a room , pin a left-over length inside a wardrobe or cupboard door so that it ages along with the paper on the walls .Wallpaper them or cover them with fabric : felt , hessian , sacking , lining fabric or printed cotton .If varnishing a door , it is better to use a 2½ or 3in brush , rather than 1 or 1½in .Paint a blown egg all over in one colour and varnish it ."
3,3,"v → Obj: Goal (NP) → CNI: Agent (None) → DNI: Theme (None)Dilute 2 UNC oz of borax in 5gals of water , using the solution to spray the foliage .Most aid specialists agree that , once the worst of a famine is over , the distribution of free food should be kept to a minimum to avoid flooding local markets .Such sights include the colorful `` tams '' -- knitted hats worn by Jamaican men to cover their mane of dreadlocks -- and the red ackee fruit that ripens at the beginning of the year ."


In [30]:
filling.select(p_all, negative=True).diagram(noncore=False)

Written 12 records.


<IPython.core.display.Javascript object>

In [31]:
cfm.display(pattern_matcher=p1_dep_NI)

In [32]:
cfm.select(p1_dep_NI).diagram(noncore=False)

Written 14 records.


<IPython.core.display.Javascript object>

In [33]:
cfm.select(p2_dep_NI).diagram(noncore=False)

Written 16 records.


<IPython.core.display.Javascript object>

In [34]:
cfm.select(p1_dep_NI).diagram(noncore=False)

Written 14 records.


<IPython.core.display.Javascript object>

In [35]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(), 
    negative=False, 
    min_count=0,
)

In [36]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p3), 
    negative=False, 
    min_count=0,
)

,freq.,Patterns
1,11,"v → Obj: Theme (NP) → Dep: Goal (PP) → CNI: Agent (None)Odd-Knut makes coffee using the method we have become familiar with -- chuck the coffee in a pot , add snow to taste , boil on the stove , when empty add coffee and/or snow as necessary .What better way to test the mettle of the new Calibra 16v than to pitch it directly against the class best , VW 's Corrado G60 ?Press each block firmly in place , and weight them with something heavy until the adhesive has set .CAPILLARY REFILL TEST : This involves pressing your thumb onto the horse 's gum .Again , change hands , push the tiller to where you were sitting , watch for the boom , as it swings across , straighten up and sit down on the new side .A TIP : If a power failure occurs ( or the mains have to be turned off ) temperature loss can be much reduced by just throwing a blanket from the bed over the aquarium .Be very careful of old fluorescent light tubes , which can explode if dropped , and never puncture old aerosol cans or throw them on a fire .Do not throw rubbish onto an open fire in the living room .Throws her bouquet to her unmarried friends before she leaves .Toss the potatoes in the butter , spoon into a warmed serving dish and garnish with more sprigs of basil .Or moving it to Lubbock or somewhere possibly is not the answer ."
2,6,"v → Obj: Theme (NP) → Dep: Path (PP) → CNI: Agent (None)I ca n't afford to keep chucking two quid down the drain .The tank blew up , hurling the Vauxhall through the closed garage doors , which caused twenty more gallons of petrol to explode , blowing the front off the house .And if your sauce has turned lumpy , press it through a sieve .` Shove it up your armpit ! ""If people want to escape urban gloom I have a good suggestion : throw your TV , video and stereo through the window , lock your car in the garage and buy a bicycle .That bit about throwing the stone up the alley to make him rush off after it , I got that from a detective story ."
3,5,"v → Obj: Theme (NP) → Dep: Source (PP) → CNI: Agent (None)I was impressed by the streamlined efficiency of the Surgicentre ( if a little dismayed by the check 'em in , chop it off , chuck 'em out attitude ) .Take him away and chuck him off the derrick . ""Do not fling me from your houseThe aim is to leap around in an inflated suit and push your opponent off the mat .Alternatively , hold a heavy medicine ball in both hands , bend your elbows and thrust it from you ."
4,3,"v → Obj: Goal (NP) → Dep: Theme (NP) → CNI: Agent (None)` Chuck me a can ! ""` Chuck us a hammer , Cyril , "" he shouted down bossily , ` the brickwork 's not too bad .Shove us the marmalade , Jo . """
5,2,"v → Obj: Theme (NP) → Dep: Goal (AVP) → CNI: Agent (None)Cameras won , and in chucking my fishing bag ashore , the 4lb breaking-strain cast snapped and the trout was gone .In other words , fire up the Transit , shove every member of the band inside , come down and make thorough , absolute pigs of yourselves ."
6,2,v → Obj: Theme (NP) → Dep: Source (PP) → Dep: Goal (PP) → CNI: Agent (None)Push empty needles from non working to working position and make a hem by placing loops of first row worked in main yarn evenly along the row .This article inaugurated Rothermere 's massive press campaign which helped to thrust the BUF from comparative obscurity into the limelight .
7,1,v → Obj: Theme (NP) → Dep: Distance (AVP) → CNI: Agent (None) → INI: Path (None)However it would be unwise to press these objections too far .
8,1,v → Obj: Theme (NP) → Dep: Path (PP) → Dep: Distance (PP) → CNI: Agent (None)Cut a small hole at the bottom point and push the green covered pencil or stick through the hole for about 2cm ( Fig 3 ) .
9,1,"v → Obj: Theme (NP) → Dep: Path (AVP) → CNI: Agent (None)Knee : Protect your groin and throat , yell , and thrust your hip forward for extra power ."
10,1,v → Obj: Theme (NP) → Dep: Result (AVP) → Dep: Manner (AVP) → CNI: Agent (None)Add to bowl and toss salad ingredients together gently .


In [37]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p1_dep_NI) | (p2_dep_NI) | (p3) | (p4) | (p5_rel_subj) | (p5_rel_obj), 
    negative=True, 
    min_count=2,
)

,freq.,Patterns
1,7,"Ext: Agent (NP) → v → Obj: Theme (NP) → INI: Goal (None)When you chuck the arrow , gravity takes effect and it arches realistically in flight .On duty she strode about looking as if she could not wait to get off duty and chuck a discus .I go to funerals and chuck earth .Morton went off with him and returned pushing a wide porter 's barrow , with empty sacks in it .In other words , there will always be diesel and petrol , however much the proponents of each fuel push their own products .She says he throws things without thinking about the damage it does .When he told them how he had thrown the Bible , Mrs Wood gave a little scream ."
2,7,"v → Dep: Goal (PP) → Obj: Theme (NP) → CNI: Agent (None)There 's a lot to be said for making the trench in autumn , chucking in household waste all winter and replacing the soil in spring .Press down the edges to seal and trim off the excess pastry .Push over those fried tomatoes , will you , James ? ""Simmer with plenty of fish sauce , of course , and perhaps some fresh ginger slices , and then remove them and throw in finely sliced lemon grass and kaffir lime leaf .Throw over a covering of compost , peat , leaves or newspaper , wrap with sacking or cartons , and hardy plants will then be happy enough , for a few weeks if need be .He reckoned he had done well to get away with two cups of tea and forty minutes of reminiscence before an opening arose to thrust in a question .All they had to do was pile the earth back on top of the coffin and toss in a few bulbs of garlic just to be on the safe side ."
3,4,"v → Dep: Goal (PP) → CNI: Agent (None) → DNI: Theme (None)Mix breadcrumbs and herbs and press onto the lamb .Using cool hands ( wash under cold running water if necessary ) , transfer the roof to the cake and press down gently .Wash the grapes and halve them , then toss into the bowl .4 M formaldehyde RNA gel and transferred to Hybond N +nylon membrane ( Amersham-Pharmacia , UK ) ."
4,3,"Ext: Agent (NP) → v → Obj: Theme (NP) → DNI: Goal (None)With hardly a glance towards the board , he flung his darts .With a triumphant yell , she hurled the harpoon .Some began to hurl stones ."
5,3,"Ext: Theme (NP) → Ext: Theme (NP) → v → Dep: Path (PP) → CNI: Agent (None)He shut the kitchen door smartly to avoid the dishcloth that was hurled in his direction .He began striding away , wishing to put as much distance as possible between himself and this nightmare that had been thrust towards him .It was just a casual phrase that was tossed back and forth between acquaintances , and meant nothing -- only she did n't want it to mean nothing ."
6,3,"Ext: Agent (NP) → v → Gen: Theme (Poss) → Obj: Goal (NP)` I have a lot of questions I have to ask you , "" he said , opening the rear door of the Ford and pushing her inside , ` but first just answer me one thing . ""He seized her and thrust her outside .She gasped as he yanked her closer ."
7,3,"Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Goal (PP)` A long time ago , "" said Weir , drinking deeply from the flask Stephen pushed over to him .Some she discarded and threw into the fire , others she kept .Do you want me to apologise for picking up the gauntlet you threw down ? """


In [38]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p1_dep_NI) | (p2_dep_NI) | (p3) | (p4) | (p5_rel_subj) | (p5_rel_obj), 
    negative=True, 
    min_count=2,
)

,freq.,Patterns
1,7,"Ext: Agent (NP) → v → Obj: Theme (NP) → INI: Goal (None)When you chuck the arrow , gravity takes effect and it arches realistically in flight .On duty she strode about looking as if she could not wait to get off duty and chuck a discus .I go to funerals and chuck earth .Morton went off with him and returned pushing a wide porter 's barrow , with empty sacks in it .In other words , there will always be diesel and petrol , however much the proponents of each fuel push their own products .She says he throws things without thinking about the damage it does .When he told them how he had thrown the Bible , Mrs Wood gave a little scream ."
2,7,"v → Dep: Goal (PP) → Obj: Theme (NP) → CNI: Agent (None)There 's a lot to be said for making the trench in autumn , chucking in household waste all winter and replacing the soil in spring .Press down the edges to seal and trim off the excess pastry .Push over those fried tomatoes , will you , James ? ""Simmer with plenty of fish sauce , of course , and perhaps some fresh ginger slices , and then remove them and throw in finely sliced lemon grass and kaffir lime leaf .Throw over a covering of compost , peat , leaves or newspaper , wrap with sacking or cartons , and hardy plants will then be happy enough , for a few weeks if need be .He reckoned he had done well to get away with two cups of tea and forty minutes of reminiscence before an opening arose to thrust in a question .All they had to do was pile the earth back on top of the coffin and toss in a few bulbs of garlic just to be on the safe side ."
3,4,"v → Dep: Goal (PP) → CNI: Agent (None) → DNI: Theme (None)Mix breadcrumbs and herbs and press onto the lamb .Using cool hands ( wash under cold running water if necessary ) , transfer the roof to the cake and press down gently .Wash the grapes and halve them , then toss into the bowl .4 M formaldehyde RNA gel and transferred to Hybond N +nylon membrane ( Amersham-Pharmacia , UK ) ."
4,3,"Ext: Agent (NP) → v → Obj: Theme (NP) → DNI: Goal (None)With hardly a glance towards the board , he flung his darts .With a triumphant yell , she hurled the harpoon .Some began to hurl stones ."
5,3,"Ext: Theme (NP) → Ext: Theme (NP) → v → Dep: Path (PP) → CNI: Agent (None)He shut the kitchen door smartly to avoid the dishcloth that was hurled in his direction .He began striding away , wishing to put as much distance as possible between himself and this nightmare that had been thrust towards him .It was just a casual phrase that was tossed back and forth between acquaintances , and meant nothing -- only she did n't want it to mean nothing ."
6,3,"Ext: Agent (NP) → v → Gen: Theme (Poss) → Obj: Goal (NP)` I have a lot of questions I have to ask you , "" he said , opening the rear door of the Ford and pushing her inside , ` but first just answer me one thing . ""He seized her and thrust her outside .She gasped as he yanked her closer ."
7,3,"Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Goal (PP)` A long time ago , "" said Weir , drinking deeply from the flask Stephen pushed over to him .Some she discarded and threw into the fire , others she kept .Do you want me to apologise for picking up the gauntlet you threw down ? """


In [39]:
cm.display(pattern_matcher=(p5_rel_obj))

,freq.,Patterns
1,5,"Obj: Theme (NP) → Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Goal (PP)One of them picked up the rolled newspaper which Clasper had flung into the crowd .He slurped as noisily as a ravenous dog , mopping up what was left with hard crusts of bread which Father Peter shoved before him on a wooden platter .The Feldwebel took it into a corner and pulled out a large envelope which he threw onto the table .He caught the pliers that she threw to him , and the ball of string , and then went to hold the step-ladder steady while Kitty descended .` We 'll be more likely to get killed with them falling on us than anything what the Jerrys throw down , "" Hetty said ."
2,1,Obj: Theme (NP) → Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Source (PP)The next article that Zach dragged out was a stiff black circular object .
3,1,Obj: Theme (NP) → Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Area (PP)She was no longer a nervous little nineteen-year-old that he could push around as he pleased .
4,1,"Obj: Theme (NP) → Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Manner (AVP) → Dep: Goal (PP)When she hurried out to the car , carrying her drawing materials which she had shoved hurriedly into a canvas bag , Julius was already sitting in the driving seat ."
5,1,Obj: Theme (NP) → Obj: Theme (NP) → Ext: Agent (NP) → v → Dep: Goal (AVP)She sank down on to the bed and glanced at the writing pad that she 'd tossed there after a brief effort to write to Arnie .


In [40]:
cm.select(p1_dep_NI).diagram(noncore=False)

Written 22 records.


<IPython.core.display.Javascript object>

In [41]:
cm.select(p2_dep_NI).diagram(noncore=False)

Written 39 records.


<IPython.core.display.Javascript object>

### Placing

In [42]:
placing = Frame('Placing')
placing.groups.data_frame['annotationSet.LU'].unique()

array(['deposit.v', 'embed.v', 'hang.v', 'immerse.v', 'implant.v',
       'inject.v', 'insert.v', 'insertion.n', 'lay.v', 'lean.v', 'load.v',
       'lodge.v', 'pack.v', 'park.v', 'pile.v', 'place.v', 'placement.n',
       'plant.v', 'position.v', 'rest.v', 'set.v', 'situate.v', 'smear.v',
       'stand.v', 'stash.v', 'stow.v', 'stuff.v', 'tuck.v', 'wrap.v',
       'emplace.v', 'enclose.v', 'bag.v', 'billet.v', 'bin.v', 'bottle.v',
       'archive.v', 'box.v', 'cage.v', 'crate.v', 'file.v', 'garage.v',
       'pocket.v', 'shelve.v', 'stable.v', 'perch.v', 'sit.v', 'daub.v',
       'drizzle.v', 'dab.v', 'drape.v', 'brush.v', 'dust.v', 'heap.v',
       'jam.v', 'rub.v', 'cram.v', 'shower.v', 'pot.v', 'sheathe.v',
       'warehouse.v', 'shoulder.v', 'stick.v', 'bestow.v', 'put.v',
       'package.v', 'plunge.v'], dtype=object)

In [43]:
placing.display(
    pattern_matcher= x,
    negative=True, 
    min_count=8,
)

In [44]:
placing.display(
    pattern_matcher= p_all,
    negative=True, 
    min_count=8,
)

In [45]:
placing.display(
    pattern_matcher= p1_all,
    negative=False, 
    min_count=8,
)

In [46]:
placing.select(p_all).diagram(noncore=True)

Written 189 records.


<IPython.core.display.Javascript object>

In [47]:
placing.select(p1_all).diagram(noncore=True)

Written 53 records.


<IPython.core.display.Javascript object>

In [48]:
placing.select(p2_all).diagram(noncore=True)

Written 49 records.


<IPython.core.display.Javascript object>

In [49]:
placing.display(
    pattern_matcher= p2_all,
    negative=False, 
    min_count=5,
)

In [50]:
placing.select(p3_all).diagram(noncore=True)

Written 74 records.


<IPython.core.display.Javascript object>

In [51]:
placing.display(
    pattern_matcher= p3_all,
    negative=False, 
    min_count=5,
)

In [52]:
placing.select(p3_obj).diagram(noncore=True)

Written 6 records.


<IPython.core.display.Javascript object>

In [53]:
placing.select(p3_dep).diagram(noncore=True)

Written 24 records.


<IPython.core.display.Javascript object>

In [54]:
placing.select(p4).diagram(noncore=True)

Written 8 records.


<IPython.core.display.Javascript object>

The end.

In [55]:
placing.select(p5_all).diagram(noncore=True)

Written 23 records.


<IPython.core.display.Javascript object>

In [56]:
bec_att = Frame('Becoming_attached')
bec_att.groups.data_frame['annotationSet.LU'].unique()

array(['stick.v', 'agglutinate.v', 'bind.v', 'fasten.v', 'moor.v',
       'attach.v'], dtype=object)

In [57]:
bec_att.select(p_all).diagram(noncore=True)

Written 12 records.


<IPython.core.display.Javascript object>

In [58]:
attaching = Frame('Attaching')
attaching.groups.data_frame['annotationSet.LU'].unique()

array(['strap.v', 'append.v', 'yoke.v', 'truss.v', 'link.v', 'connect.v',
       'join.v', 'mount.v', 'fasten.v', 'attach.v', 'tie.v', 'tack.v',
       'glue.v', 'staple.v', 'tape.v', 'pin.v', 'nail.v', 'affix.v',
       'chain.v', 'paste.v', 'sew.v', 'stick.v', 'lash.v', 'bond.v',
       'cinch.v', 'secure.v', 'fuse.v', 'shackle.v', 'bind.v', 'detach.v',
       'manacle.v', 'handcuff.v', 'rivet.v', 'tether.v', 'cement.v',
       'gum.v', 'fix.v', 'weld.v', 'solder.v', 'moor.v', 'anchor.v',
       'attachment item.n', 'hitch.v', 'tying.n', 'untie.v', 'hook.v',
       'plaster.v', 'attachment act.n'], dtype=object)

In [59]:
attaching.select(p_all).diagram(noncore=True)

Written 124 records.


<IPython.core.display.Javascript object>

In [60]:
attaching.select(p1_all).diagram(noncore=True)

Written 39 records.


<IPython.core.display.Javascript object>

In [61]:
attaching.display(
    pattern_matcher= p1_all,
    negative=False, 
    min_count=3,
)

In [62]:
attaching.select(p2_all).diagram(noncore=True)

Written 20 records.


<IPython.core.display.Javascript object>

In [63]:
attaching.display(
    pattern_matcher= p2_all,
    negative=False, 
    min_count=3,
)